# Analysis of Deutsh's algorithm

Deutsch's Problem

Input: a function f: {0,1} -> {0,1}
Output 0 if f is constant, 1 if f is balanced



In [4]:
from qiskit import QuantumCircuit

def deutsch_function(case: int):
    #generate a valid deutsch function as a quantum circuit
    if case not in [1,2,3,4]:
        raise ValueError("`case` must be 1,2,3, or 4.")
    f = QuantumCircuit(2)
    if case in [2, 3]:
        f.cx(0, 1)
    if case in [3, 4]:
        f.x(1)
    return f
display(deutsch_function(3).draw())

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

In [5]:
def compile_circuit(function: QuantumCircuit):
    #compiles circuit for use in Deutsch's algorithm
    n = function.num_qubits - 1
    qc = QuantumCircuit(n+1, n)

    qc.x(n)
    qc.h(range(n+1))

    qc.barrier()
    qc.compose(function, inplace=True)
    qc.barrier()

    qc.h(range(n))
    qc.measure(range(n), range(n))

    return qc

display(compile_circuit(deutsch_function(3)).draw())

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

In [10]:
from qiskit_aer import AerSimulator

def deutsch_algorithm(function: QuantumCircuit):
    #determine if deutsch function is constant or balanced
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if measurements[0] == "0":
        return "constant"
    return "balanced"
f = deutsch_function(4)
display(f.draw())
display(deutsch_algorithm(f))

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

'constant'